In [3]:
from model.international_game import InternationalGame

In [4]:
current_game = InternationalGame(1, None, None, None)


In [5]:
current_game.init()

In [6]:
from copy import deepcopy

import numpy as np

import ai.config as config
from model.game import Action
from ai.agent import AlphaZero


import ai.utils as ut
import ai.modified_tree_search as mts
import ai.config as config

In [7]:
from model.international_game import InternationalGame

In [8]:
best_model = ut.load_best_model()

loading version 1


In [9]:
current_model = ut.load_best_model()

loading version 1


In [10]:
current_agent = AlphaZero(config.MCTS_SIMS)
best_agent = AlphaZero(config.MCTS_SIMS)

In [11]:
class RejectedActionError(Exception):
    pass

In [12]:
import random
random.seed(101)

In [13]:
import numpy as np
np.random.seed(101)

In [14]:
def self_play(agent: AlphaZero, model):
    current_game = InternationalGame(1, None, None, None)
    sample_builder = ut.SampleBuilder()
    
    current_game.init()
    agent.build_mct(ut.GameState(current_game), model)
    
    turn = 1
    while not current_game.end():
        if turn % 20 == 0:
            print()
        tau = 1 if turn < config.TURNS_UNTIL_TAU0 else 0
        
        action, state_stack, value, pi = agent.train_act(tau)
        
        if not current_game.is_legal_action(action):
            print(current_game.grid)
            print('------------------------------')
            print(agent.mct.root.game_state.get_game().grid)
            print(action)
            print(agent.mct.action_encoder.transform([ut.to_label(action)]))
            actions = current_game.get_all_possible_actions()
            actions = list(map(ut.to_label, actions))
            print(actions)
            print(agent.mct.root.edges)
            raise RejectedActionError('shit')
        
        current_game.apply_action(action)
        if state_stack.head is None:
            return
        sample_builder.add_move(state_stack, pi)
        
        agent.on_update(action)
        
        turn += 1
        print('*', end='')
    
    value = ut.evaluate(current_game)
    
    sample_builder.commit_sample(value, config.MAXIMIZER)
    
    return sample_builder

In [15]:
import time
start_time = time.monotonic()
self_play(current_agent, current_model)
print(f'slept for {time.monotonic() - start_time}')

*******************
********************
********************
********************
********************
********************
***slept for 414.6710000000021


In [ ]:
import time
import random

from IPython.display import display
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import cufflinks as cf
import plotly.graph_objects as go

In [ ]:
init_notebook_mode(connected=True)
cf.go_offline()
np.random.seed(5)

In [ ]:
await task

In [ ]:
print(len(mct.root.edges))

In [ ]:
currentAgent = AlphaZero(config.MCTS_SIMS)


In [ ]:
currentAgent.build_mct(ut.GameState(current_game), current_model)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:


class PlayGround:
    def __init__(self):
        self.action_space_shape = len(get_action_space(10, 10))
        self.current_model = NeuralNetwork(config.REG_CONST, config.LEARNING_RATE, 
                                      (10,10,25),   self.action_space_shape, config.HIDDEN_CNN_LAYERS)

        #self.best_model = NeuralNetwork(config.REG_CONST, config.LEARNING_RATE, 
                                   #(10,10,25),   self.action_space_shape, config.HIDDEN_CNN_LAYERS)
        
        self.best_model.model.set_weights(self.current_model.model.get_weights())
        self.exp = []
        self.turn = []
        
        self.fig = go.FigureWidget()
        self.fig.update_layout(xaxis_title="Turn", yaxis_title='Expected value')
        #self.fig.add_scatter(fill='tozeroy')
        self.fig.add_scatter()

    def plot_figure(self):
        display(self.fig)
        
    def update_plot(self):
        with self.fig.batch_update():
            self.fig.data[0].x = self.turn
            self.fig.data[0].y = self.exp
            
    def apply_and_update(self, game, playing_agent: AlphaZero, other_agent: AlphaZero, action: Action, action_id: int):
        print(game.grid)
        print(action)
        if not game.is_legal_action(action):
            print(game.current_turn)
            actions = game.get_all_possible_actions()
            for act in actions:
                print(act)
            raise RejectedAction("your fucked bro!")
        game.apply_action(action)
        playing_agent.update_root(action_id)
        if other_agent is not None:
            other_agent.update_root(action_id)
            
        #return sample_builder

    def self_play(self):
        current_Agent = AlphaZero(config.MCTS_SIMS, config.CPUCT, self.current_model, pov=0, name="Current AlphaZero")
        
        game = InternationalGame(1, None, None, None)
        sample_builder = SampleBuilder()

        for ep in range(config.EPISODES):
            game.init()
            current_Agent.build_mcts(StateStack(deepcopy(GameState(game))))

            turn = 1
            player: AlphaZero = current_Agent

            while not game.end():
                player.pov = game.currentTurn
                tau = 1 if turn < config.TURNS_UNTIL_TAU0 else 0
                try:
                    action, action_id, state_stack, value, pi = player.train_act(tau)
                except KeyError as e:
                    print(e)
                    print(player.mcts.root.edges)

                if player.pov == 1:
                    self.turn.append(turn)
                    self.exp.append(value)
                self.apply_and_update(game, current_Agent, None, action, action_id)
                self.update_plot()
                sample_builder.commit_move(state_stack, pi)
                turn += 1
                print('*', end='')

            value = evaluate(game)
            sample_builder.commit_sample(value, config.MAXIMIZER)

        return sample_builder
    
    def fit(self, sample_builder:SampleBuilder):
        for i in range(10):
            minibatch = random.sample(sample_builder.samples, min(config.BATCH_SIZE, len(sample_builder.samples)))
            training_states = np.array([row['state'].get_deep_representation_stack() for row in minibatch])
            training_targets = {'value_head': np.array([row['value'] for row in minibatch]),
                                'policy_head': np.array([row['policy'] for row in minibatch])}

            ret = self.current_model.fit(training_states, training_targets, epochs=config.EPOCHS, verbose=1, validation_split=0, batch_size = 32)

In [ ]:
playground = PlayGround()

In [ ]:
#playground.plot_figure()

In [ ]:
try:
    playground.self_play()
except RejectedAction as e:
    print(e)

In [ ]:
#calc_new_rate(1613, [1609, 1477, 1388, 1586, 1720], [2, 0, 1, 1, 2])
#1601

In [ ]:
import numpy as np
import random
        

In [ ]:
import ai.figures as figs

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import cufflinks as cf
import ai.figures as figs
init_notebook_mode()
cf.go_offline()

In [ ]:
import pandas as pd
df = pd.read_csv('data/users v3.csv')
df.index.name = 'player_id'

In [ ]:
iplot(figs.registration_intensity_histogram(df))

In [ ]:
iplot(figs.categories_bars(df))
iplot(figs.categories_ratio_pie(df))

In [ ]:
df.head()

In [ ]:
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['year'] = df['signup_date'].apply(lambda t: t.year)
df['month'] = df['signup_date'].apply(lambda t: t.month)
df['day'] = df['signup_date'].apply(lambda t: t.day)
df.head()

In [ ]:
bymonth = df.groupby('month').count()

trace1 = go.Scatter(x = bymonth.index,y=bymonth.rate,
                          marker = dict(color = 'rgba(31, 119, 180, 0.8)',
                                        line=dict(color='rgb(0,0,0)',
                                                  width=1.5)))
layout = dict(barmode = 'group',title = 'Registration Intensity',
                 xaxis= dict(title= 'Month',ticklen= 5,zeroline= False, ticks="inside"), 
                 yaxis = dict(title = 'Users', ticklen = 5, zeroline = False, ticks="outside"))
data = [trace1]

fig = {'data': data, 'layout': layout}
    
iplot(fig)